In [ ]:
# import os
# import numpy as np
# import nibabel as nib
# from PIL import Image

# def save_slices_to_nii(image_folder, output_folder, original_nii_path):
#     slices = []
    
#     # 从原始文件中获取头部和仿射矩阵信息
#     original_nii = nib.load(original_nii_path)
#     affine = original_nii.affine
#     header = original_nii.header

#     # 按文件名排序确保图像是按正确的顺序加载的
#     filenames = sorted(os.listdir(image_folder))

#     # 遍历子文件夹中的所有.png图片
#     for filename in filenames:
#         if filename.endswith('.png'):
#             img_path = os.path.join(image_folder, filename)
#             img = Image.open(img_path)
            
#             # Resize the image to 512x512
# #             img = img.resize((512, 512), Image.LANCZOS)

#             img_array = np.array(img)
#             slices.append(img_array)

#     # 将所有的图片堆叠起来形成一个3D数组
#     combined_data = np.stack(slices, axis=-1)

#     # 使用 nibabel 创建 NIfTI 图像并保存
#     nii_img = nib.Nifti1Image(combined_data, affine, header=header)
#     output_path = os.path.join(output_folder, f'{os.path.basename(image_folder)}.nii.gz')
#     nib.save(nii_img, output_path)

# # 文件夹路径
# input_folder = "/home/xulei/projects/wei/2DSAM3D/LUNA16/SAM_label"
# output_folder = "/home/xulei/projects/wei/2DSAM3D/LUNA16/SAM_label_3d"
# original_nii_path = "/home/xulei/projects/wei/2DSAM3D/LUNA16/Trimage"

# # 如果输出文件夹不存在，则创建
# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)

# # 遍历文件夹并处理每个子文件夹
# for foldername in os.listdir(input_folder):
#     current_folder_path = os.path.join(input_folder, foldername)
#     if os.path.isdir(current_folder_path):
#         save_slices_to_nii(current_folder_path, output_folder, original_nii_path)


In [5]:
import os
import nibabel as nib
from PIL import Image
import numpy as np
import re
from tqdm import tqdm
from collections import defaultdict
import concurrent
from concurrent.futures import ThreadPoolExecutor

def process_group(base_name, files, image_folder, original_nii_folder):
    slices = []

    # 从原始文件中获取头部和仿射矩阵信息
    original_nii_path = os.path.join(original_nii_folder, f"{base_name}.nii.gz")
    original_nii = nib.load(original_nii_path)
    affine = original_nii.affine
    header = original_nii.header

    # 遍历文件夹中的所有.png图片
    for filename in files:
        img_path = os.path.join(image_folder, filename)
        img = Image.open(img_path)
        img=img.resize((512, 512), Image.LANCZOS)
        img_array = np.array(img)
        slices.append(img_array)

    # 将所有的图片堆叠起来形成一个3D数组
    combined_data = np.stack(slices, axis=-1)

    # 使用 nibabel 创建 NIfTI 图像并保存
    nii_img = nib.Nifti1Image(combined_data, affine, header=header)
    output_path = os.path.join(output_folder, f"{base_name}.nii.gz")
    nib.save(nii_img, output_path)

def save_slices_to_nii(image_folder, output_folder, original_nii_folder):
    # 获取所有.png文件的名字，并按照id排序
    filenames = sorted(os.listdir(image_folder), key=lambda x: int(re.search(r"_(\d+).png", x).group(1)))

    # 将文件名按照 IMG_xxxx 分组
    grouped_files = defaultdict(list)
    for filename in filenames:
        base_name = re.search(r"(IMG_\d+)_", filename).group(1)
        grouped_files[base_name].append(filename)

    # 创建一个ThreadPoolExecutor
    with ThreadPoolExecutor() as executor:
        # 使用executor.map来并行处理每个图片组
        futures = [executor.submit(process_group, base_name, files, image_folder, original_nii_folder) 
                   for base_name, files in grouped_files.items()]

        # 使用tqdm和as_completed来显示进度条
        for _ in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
            pass

# 文件夹路径
input_folder = "/home/xulei/projects/wei/2DSAM3D/SAM_label"
output_folder = "/home/xulei/projects/wei/2DSAM3D/SAM_label_3d"
original_nii_folder = "/home/xulei/projects/wei/2DSAM3D/LUNA16/Trimage"

# 如果输出文件夹不存在，则创建
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 处理输入文件夹
save_slices_to_nii(input_folder, output_folder, original_nii_folder)


100%|██████████| 532/532 [12:23<00:00,  1.40s/it]
